In [1]:
import re
from pathlib import Path

import spacy, pandas as pd, gensim as gn
import gensim.corpora as corpora
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaMulticore, LdaModel
# from gensim.models.ldamodel import LdaModel

# spacy internal setup
if not 'nlp' in locals():
    model_loaded: bool = False
    while not model_loaded:
        # we keep trying until this works...
        try:
            print("attempting to load model")
            # attempt to load the spacy model
            nlp: spacy.Language = spacy.load("en_core_web_sm")
            print("model loaded successfully")
            model_loaded = True
        except BaseException as e:
            print("model failed to load")
            # if we fail to load the model, we are going to make
            # sure the package is installed...
            from spacy.cli import download
            download("en_core_web_sm")
            # then loop around and try to load it again

attempting to load model
model loaded successfully


In [2]:
path_root: Path = Path(".").resolve()

In [3]:
md_text_raw: str = path_root.joinpath("moby_dick.txt").read_text('utf-8')

# Preprocess the text

In [4]:
# DEFINE SOME USEFUL VARIABLES FOR PREPROCESSING
# utf-8 codes for some characters in the text
dq_op: str = "\u201C" # double quotes open
dq_cl: str = "\u201D" # double quotes close
sq_op: str = "\u2018" # single quote open
sq_cl: str = "\u2019" # single quote close
under: str = "\u005F" # underscore
hyphe: str = "\u002D" # hyphen/minus
mdash: str = "\u2014" # em dash

In [5]:
# replace newlines and carriage returns with whitespace
md_text: str = re.sub("[\n\r]", " ", md_text_raw)
# remove everything before the first chapter
md_text = re.sub("[\w\W]*(?=CHAPTER 1\. Loomings)", "", md_text, count=1)
# remove everything after the end of the epilogue
md_text = re.sub("\s*(\*\*\* END)[\w\W]+", "", md_text, count=1)
# there are some weird embedded books in the middle of the text
# first we remove the in-betweens of the embedded books
md_text = re.sub("(Thus ends BOOK[\w\W]+?)([\w\W]+?BOOK[\w\W]+?)(?=\s\s)", "", md_text)
# then we get rid of the chapter headings of the embedded books
md_text = re.sub("BOOK\s+[IV]+([\w\W]+?(?=CHAPTER)CHAPTER\s+[IV\d]+\.\s+[\w\W]+?(?=\.)\.)?", "", md_text)
# CLEANING UP IMPORTANT UTF-8 CHARACTERS
# replace all utf single quotes with ascii single quotes
md_text = re.sub(f"{sq_op}|{sq_cl}", "\'", md_text)
# replace all utf double quotes with ascii double quotes
md_text = re.sub(f"{dq_op}|{dq_cl}", "\"", md_text)
# replace utf underscores with ascii
md_text = re.sub(f"{under}", "_", md_text)
# replace utf hyphens with ascii
md_text = re.sub(f"{hyphe}", "-", md_text)
# replace utf em dash with hyphen
md_text = re.sub(f"{mdash}", "-", md_text)
# next (and I don't know if this is a good idea...) we're going to replace
# all punctuation that is NOT a:
# . or ! or ? or whitespace or "
md_text = re.sub("[^\w\d\s\.\"?!]", " ", md_text)
# finally, we want to replace repeated whitespace with single whitespace
md_text = re.sub("\s+", " ", md_text)

In [6]:
# defining a function to contain intermediate variables
def split_chapters(text: str) -> dict[str, str]:
    # build a list of the chapter titles and their contents
    list_results: list[str] = [result.strip() for result in re.split("(CHAPTER\s\d+\.\s)|(Epilogue)", text) if result != "" and result is not None]
    # use that list to create a dictionary that is {chapter_title: chapter_content}
    return {x: list_results[i+1] for i, x in enumerate(list_results) if re.match("(CHAPTER\s\d+\.)|(Epilogue)", x)}
# run the processed text through the chapter splitter
dict_chapters: dict[str, str] = split_chapters(md_text)
# dict_chapters

In [7]:
# instantiate the spacy nlp object
# this object contains all of the functionality for
# turning a string or list of strings into spacy "Documents"
"""we no longer need to do the following line as it's loaded with the imports"""
# nlp: spacy.Language = spacy.load("en_core_web_sm")

# we CAN add pipelines here though
nlp.add_pipe("merge_noun_chunks")

# our book is a little too long for the default
# processing limit so we increase it slightly
nlp.max_length = 1200000

In [8]:
# let spacy process the full text
# this produces a "Document"
# the document is a tokenized representation of the full
# text with a bunch of extra information attached to the tokens
doc = nlp(md_text)

In [9]:
# spacy stores things like parts of speech as integer tags
# we can use this reverse lookup table to get the string label
lookup: dict[int, str] = {y: x for x, y in spacy.symbols.IDS.items()}

In [10]:
# we can now build a list of all things that are like nouns for the full text
noun_alikes: list[str] = [x.text for x in doc if lookup[x.pos] in {"NOUN", "PROPN", "PRON"}]

In [11]:
# using pandas we build a series from the noun_alikes list
nouns: pd.Series = pd.Series(noun_alikes)
# that we can use to get the unique nouns and how many times they appear in the text
nouns_counts: pd.Series = nouns.value_counts()

In [12]:
nouns_counts

it                2187
I                 2074
he                1643
him               1054
that               863
                  ... 
one voyage           1
the logs             1
many hunters         1
given waters         1
another orphan       1
Name: count, Length: 27214, dtype: int64

In [13]:
# bigram_mod = gensim.models.phrases.Phraser()

In [14]:
[x for x in list(doc.sents)[0].as_doc()]

[CHAPTER, 1, .]

In [15]:
# culling the tokens to get rid of the punctuation tokens and stopwords.
tokens_filtered: list[str] = [
    token.lemma_ for token in doc if 
    not token.is_stop
    and not token.is_punct
    and not token.is_digit
]

In [16]:
texts: list[list[str]] = [
    [
        token.text for token in sentence.as_doc() if
        not token.is_stop
        and not token.is_punct
        and not token.is_digit
    ]
    for sentence in doc.sents
]

In [17]:
id2word = corpora.Dictionary(texts)

In [18]:
corpus = [id2word.doc2bow(text) for text in texts]

In [19]:
lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=20, 
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto',
    per_word_topics=True
)

In [20]:
doc_lda = lda_model[corpus]

In [24]:
# lda_model.print_topics()

In [22]:
import pyLDAvis as vis
import pyLDAvis.gensim_models as gensimvis
vis.enable_notebook()

In [23]:
gensimvis.prepare(lda_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.093777  0.066160       1        1  13.006061
3      0.002425 -0.048577       2        1  12.520831
19     0.084719  0.075764       3        1   8.920304
8      0.001562 -0.025202       4        1   7.795417
11     0.001085 -0.015836       5        1   7.140370
9      0.000762 -0.010438       6        1   6.443993
16     0.000815 -0.011272       7        1   6.131247
14     0.000889 -0.012465       8        1   5.162874
1      0.000398 -0.005079       9        1   5.056550
18     0.000373 -0.004728      10        1   4.763272
0      0.000413 -0.005279      11        1   4.488922
7      0.000232 -0.002824      12        1   4.209278
4      0.000126 -0.001445      13        1   3.035209
15     0.000448 -0.005768      14        1   2.869046
6      0.000023 -0.000114      15        1   2.189766
5     -0.000085  0.001259      16        1   1.842548
10     0.000179 -0.002121      17        1   1.841458
13    -0.000049  0.000803      18        1   1.331550
17    -0.000268  0.003581      19        1   0.627407
12    -0.000268  0.003581      20        1   0.623898, topic_info=         Term         Freq        Total Category  logprob  loglift
107      like  2182.000000  2182.000000  Default  30.0000  30.0000
534   the sea   833.000000   833.000000  Default  29.0000  29.0000
62   the ship   481.000000   481.000000  Default  28.0000  28.0000
76      round   285.000000   285.000000  Default  27.0000  27.0000
304       saw   425.000000   425.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
17    thought     0.014022   232.393225  Topic20 -10.4355  -4.6386
18      years     0.014022   168.690047  Topic20 -10.4355  -4.3182
20    driving     0.014022     1.854192  Topic20 -10.4355   0.1924
27    account     0.014022    22.736573  Topic20 -10.4355  -2.3141
28   bringing     0.014022    10.330187  Topic20 -10.4355  -1.5252

[826 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
184       4  0.982686     Let
96       18  0.948794    Look
205      10  0.960545     Yes
477       3  0.958458   a man
354      12  0.853738  a sort
...     ...       ...     ...
455       9  0.967606    view
257       7  0.734880   visit
196       4  0.986466   water
456       9  0.992015     way
18        2  0.989981   years

[131 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 20, 9, 12, 10, 17, 15, 2, 19, 1, 8, 5, 16, 7, 6, 11, 14, 18, 13])